In [1]:
import sys
sys.path.insert(1, '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib')
import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import math 

from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor


**Simulated data with true effect is 100**

In [2]:
true_effect = 120
data = dowhy.datasets.linear_dataset(beta=true_effect,
        num_common_causes=8,
        num_instruments = 0,
        num_samples=20000,
        treatment_is_binary=True)
df = data["df"]

In [3]:
df['v0'] = [1 if x == True else 0 for x in df['v0']]

In [4]:
df.head()

,W0,W1,W2,W3,W4,W5,W6,W7,v0,y
0,-1.759263,-1.854437,-1.220480,0.368100,1.734491,-2.477646,2.042240,-0.500637,0,-151.243603
1,-1.453438,-2.865878,-3.038242,-0.718792,-0.094491,-0.940804,1.185255,-0.716767,0,-361.538498
2,-1.139325,0.604992,-1.839243,-0.980263,-1.500909,0.141364,1.079599,-0.587273,0,-175.477416
3,-1.040631,-1.068341,-0.552094,-0.253964,0.381098,-1.446558,1.366093,-2.246166,0,-133.204513
4,-0.766616,-1.269039,1.116350,-0.821242,-0.249904,-1.450722,1.244098,-1.211815,0,-120.417385


## Summary
$$p_1 = E[A = 1 | X]$$
$$p_0 = E[A = 0 | X]$$

$$q_1 = \frac{\delta p_0}{\delta p_0 + p_1}$$
$$\Psi (\delta ) = q1*\mathbb{E}[Y | X,A=1] + q_0*\mathbb{E}[Y | X,A=0]$$

**Influence function**

$$\Psi (\delta ) = q_1*\phi_1 + q_0*\phi_0 + \frac{\delta*(\mu(X,1) - \mu(X,0))}{(\delta*p_1 + p_0)^2}$$ 
$$\phi_a = \frac{\mathbb{1}(A=a)}{\pi(a|x)}(Y - \mu(X,A)) +  \mu(X,a)$$

## Propensity Score 

In [5]:
logreg = LogisticRegression()
logreg.fit(df[data['common_causes_names']], df[data['treatment_name'][0]])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [6]:
df['ps_1'] = logreg.predict_proba(df[data['common_causes_names']])[:,1]
df['ps_0'] = logreg.predict_proba(df[data['common_causes_names']])[:,0]


## K-nearest neighbor

In [7]:
att, atc, est= pse.propensity_score_estimator_k_nearest_neighbor(df, data['treatment_name'][0], data['outcome_name'], 'ps_1')

In [8]:
print("Average effect on treated: {}".format(att))
print("Average effect on control: {}".format(atc))
print("Average effect: {}".format(est))
print("MAE with ground truth {}".format(abs(est - true_effect)))

Average effect on treated: 275.32040279619133
Average effect on control: 271.99881528253536
Average effect: 272.18090609141257
MAE with ground truth 152.18090609141257


## Incremental score 

**Incremental propensity score:**
$$q_1 = \frac{\delta p_0}{\delta p_0 + p_1}$$


In [9]:
delta = 2
df['incre_ps_1'] = (delta * df['ps_1']) / (delta * df['ps_1'] + df['ps_0'])
df['incre_ps_0'] = 1 - df['incre_ps_1']

In [10]:
df.head()

,W0,W1,W2,W3,W4,W5,W6,W7,v0,y,ps_1,ps_0,incre_ps_1,incre_ps_0
0,-1.759263,-1.854437,-1.220480,0.368100,1.734491,-2.477646,2.042240,-0.500637,0,-151.243603,5.839007e-18,1.0,1.167801e-17,1.0
1,-1.453438,-2.865878,-3.038242,-0.718792,-0.094491,-0.940804,1.185255,-0.716767,0,-361.538498,2.639727e-22,1.0,5.279454e-22,1.0
2,-1.139325,0.604992,-1.839243,-0.980263,-1.500909,0.141364,1.079599,-0.587273,0,-175.477416,1.145077e-08,1.0,2.290155e-08,1.0
3,-1.040631,-1.068341,-0.552094,-0.253964,0.381098,-1.446558,1.366093,-2.246166,0,-133.204513,6.940119e-19,1.0,1.388024e-18,1.0
4,-0.766616,-1.269039,1.116350,-0.821242,-0.249904,-1.450722,1.244098,-1.211815,0,-120.417385,1.153859e-16,1.0,2.307719e-16,1.0


**Regress Y on X and A:**

$$\mu = \mathbb{E}[Y | X,A]$$

In [11]:
features = data['common_causes_names'] + data['treatment_name']
rf = RandomForestRegressor(n_estimators = 400, random_state = 1)
rf.fit(df[features], df['y'])

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=400, n_jobs=None, oob_score=False,
                      random_state=1, verbose=0, warm_start=False)

In [12]:
df['predicted_y'] = rf.predict(df[features])

**Estimator:**
$$\Psi (\delta ) = q1*\mathbb{E}[Y | X,A=1] + q_0*\mathbb{E}[Y | X,A=0]$$


In [13]:
## Compute counterfactual outcome with no treatment
df_pos = df.copy()
df_pos['v0'] = 0
df['treated_cf_outcome'] = rf.predict(df_pos[features])

## Compute counterfactual outcome with treatment
df_control = df.copy()
df_control['v0'] = 1
df['control_cf_outcome'] = rf.predict(df_control[features])


In [14]:
df.head()

,W0,W1,W2,W3,W4,W5,W6,W7,v0,y,ps_1,ps_0,incre_ps_1,incre_ps_0,predicted_y,treated_cf_outcome,control_cf_outcome
0,-1.759263,-1.854437,-1.220480,0.368100,1.734491,-2.477646,2.042240,-0.500637,0,-151.243603,5.839007e-18,1.0,1.167801e-17,1.0,-152.516600,-152.516600,235.198660
1,-1.453438,-2.865878,-3.038242,-0.718792,-0.094491,-0.940804,1.185255,-0.716767,0,-361.538498,2.639727e-22,1.0,5.279454e-22,1.0,-324.139064,-324.139064,146.413719
2,-1.139325,0.604992,-1.839243,-0.980263,-1.500909,0.141364,1.079599,-0.587273,0,-175.477416,1.145077e-08,1.0,2.290155e-08,1.0,-182.638778,-182.638778,123.060293
3,-1.040631,-1.068341,-0.552094,-0.253964,0.381098,-1.446558,1.366093,-2.246166,0,-133.204513,6.940119e-19,1.0,1.388024e-18,1.0,-129.926733,-129.926733,232.800511
4,-0.766616,-1.269039,1.116350,-0.821242,-0.249904,-1.450722,1.244098,-1.211815,0,-120.417385,1.153859e-16,1.0,2.307719e-16,1.0,-121.716328,-121.716328,262.352016


In [15]:
df['incre_effect'] = df['incre_ps_1']*df['treated_cf_outcome'] + df['incre_ps_0']*df['control_cf_outcome']

In [16]:
np.mean(df['incre_effect'])

In [17]:
print("MAE with ground truth {}".format(abs(np.mean(df['incre_effect']) - true_effect)))


MAE with ground truth 30.695730675626777


**Influence function** $\mathbb{1}$

$$\Psi (\delta ) = q_0*\phi_0 + q_1*\phi_1 + \frac{\delta*(\mu(X,1) - \mu(X,0)*(A - p_1)}{(\delta*p_0 + p_1)^2}$$ 

$$\phi_a = \frac{\mathbb{1}(A=a)}{\pi(a|x)}(Y - \mu(X,A)) +  \mu(X,a)$$

In [18]:
# Y - mu(X,A)
residual = df[data['outcome_name'][0]] - df['predicted_y']

m0 = [0 if df.loc[i, 'v0'] == 1 else 1/df.loc[i, 'ps_0'] for i in range(len(df))]
m1 = [0 if df.loc[i, 'v0'] == 0 else 1/df.loc[i, 'ps_1'] for i in range(len(df))]
m0 = np.array(m0)
m1 = np.array(m1)

treated_phi = m0 * residual + df['treated_cf_outcome']
control_phi = m1 * residual + df['control_cf_outcome']

# phi_0*w0 + phi_1*w1
eff1 = df['incre_ps_1']*treated_phi + df['incre_ps_0']*control_phi

# calculate influence
a = delta*(df['treated_cf_outcome'] - df['control_cf_outcome'])*(df['v0'] - df['ps_1'])
b = (delta*df['ps_1'] + df['ps_0'])**2

eff2 = a / b

In [19]:
eff = eff1 + eff2